In [10]:
!pip install -r ./src/requirements.txt

     |████████████████████████████████| 75 kB 5.7 MB/s  eta 0:00:01


## Test the code locally

In [ ]:
!python -um main --network keras_models/lstm.py --dim 128 --timestep 1.0 --depth 1 --dropout 0.3 --mode train --batch_size 8 --partition none --train "/home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/train" --test "/home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/test" --listfile "/home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/" --model_dir mimic3models/ventilator

Using TensorFlow backend.

Namespace(batch_norm=False, batch_size=8, beta_1=0.9, deep_supervision=False, depth=1, dim=128, dropout=0.3, epochs=50, imputation='previous', l1=0.1, l2=0.1, listfile='/home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/', load_state='', lr=0.001, mode='train', model_dir='mimic3models/ventilator', network='keras_models/lstm.py', normalizer_state=None, optimizer='adam', partition='none', prefix='', rec_dropout=0.0, save_every=1, size_coef=4.0, small_part=False, test='/home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/test', timestep=1.0, train='/home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/train', verbose=2)
listfile is:::: /home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/train_listfile.csv
listfile is:::: /home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/val_listfile.csv
config_path~~~~~~~~~~~~~ /home/ec2-user/SageMaker/container/resources/discretizer_config.json
normalizer_state : /home/ec2-user/SageMaker/conta

## Test the code in SageMaker 

## Upload the data to S3

In [4]:
import boto3
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()
bucket="mimic-data-for-demo"


arn:aws:iam::468574038824:role/service-role/AmazonSageMaker-ExecutionRole-20210421T150707


train_path_s3 = sess.upload_data(
    path='/home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/train',  # source
    bucket=bucket,
    key_prefix='lstm_training/train'  # destination path in S3
)

test_path_s3 = sess.upload_data(
    path='/home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/test',  # source
    bucket=bucket,
    key_prefix='lstm_training/test'  
)

testlsfile_path_s3 = sess.upload_data(
    path='/home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/test_listfile.csv',  # source
    bucket=bucket,
    key_prefix='lstm_training/listfile'  
)
trainls_path_s3 = sess.upload_data(
    path='/home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/train_listfile.csv',  # source
    bucket=bucket,
    key_prefix='lstm_training/listfile'  
)
valls_path_s3 = sess.upload_data(
    path='/home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/val_listfile.csv',  # source
    bucket=bucket,
    key_prefix='lstm_training/listfile'  
)

In [12]:
train_path_s3

's3://mimic-data-for-demo/lstm_training/listfile/train_listfile.csv'

In [5]:
import sagemaker
from sagemaker.tensorflow import TensorFlow as TensorFlowEstimator

hyperparameters = {"network":'./keras_models/lstm.py', "batch_size": 8, "dim": 128, "timestep": 1.0, "depth":1,"dropout":0.3,"partition":'none'  }

In [6]:
inputs = { "train": 's3://mimic-data-for-demo/lstm_training/train', "test": 's3://mimic-data-for-demo/lstm_training/test', "listfile":"s3://mimic-data-for-demo/lstm_training/listfile" }
print(inputs)

{'train': 's3://mimic-data-for-demo/lstm_training/train', 'test': 's3://mimic-data-for-demo/lstm_training/test', 'listfile': 's3://mimic-data-for-demo/lstm_training/listfile'}


In [ ]:
#!python -um main --network keras_models/lstm.py --dim 128 --timestep 1.0 --depth 1 --dropout 0.3 --mode train --batch_size 8 --partition none --train "/home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/train" --test "/home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/test" --listfile "/home/ec2-user/SageMaker/mimic3-benchmarks/data/ventilator/" --model_dir mimic3models/ventilator

In [12]:
env = {
    'SAGEMAKER_REQUIREMENTS': 'requirements.txt', # path relative to `source_dir` below.
}
estimator = TensorFlowEstimator(
    framework_version="1.14",
    py_version="py3",
    instance_count=1,
    instance_type="ml.p3.2xlarge",
    role=role,
    entry_point="main.py",
    source_dir="src",
    distributions={ "parameter_server": { "enabled": True } },
    hyperparameters=hyperparameters,
    metric_definitions=[
        { "Name": "loss", "Regex": "loss: ([0-9\\.]+)" },
        { "Name": "accuracy", "Regex": "acc: ([0-9\\.]+)" },
        { "Name": "validation:auroc", "Regex": "val_auroc=([0-9\\.]+)" },
        { "Name": "validation:auprc", "Regex": "val_auprc=([0-9\\.]+)" }
    ],
    base_job_name="lstm-mimic",
    max_run=20*60,  # Maximum allowed active runtime
    use_spot_instances=True,  # Use spot instances to reduce cost
    max_wait=30*60,  # Maximum clock time (including spot delays)
)

estimator.fit(inputs)

distributions has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2021-05-27 07:28:12 Starting - Starting the training job...
2021-05-27 07:28:35 Starting - Launching requested ML instancesProfilerReport-1622100466: InProgress
......
2021-05-27 07:29:35 Starting - Preparing the instances for training............
2021-05-27 07:31:35 Downloading - Downloading input data.....................
2021-05-27 07:34:56 Training - Downloading the training image/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/s

UnexpectedStatusException: Error for Training job lstm-mimic-2021-05-27-07-27-46-550: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/usr/local/bin/python3.6 main.py --batch_size 8 --depth 1 --dim 128 --dropout 0.3 --model_dir s3://sagemaker-eu-west-2-468574038824/lstm-mimic-2021-05-27-07-27-46-550/model --network ./keras_models/lstm.py --partition none --timestep 1.0"